In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = None

In [3]:
cwd = os.getcwd()

create_forecast_basic_folder_path = os.path.dirname(cwd)

sys.path.append(create_forecast_basic_folder_path)

In [4]:
from global_functions import get_forecast_version_folder_location, get_newest_date_file

In [5]:
forecast_version_folder_location=get_forecast_version_folder_location(r'{}\background_files\forecast_version_folder_location.txt'.format(create_forecast_basic_folder_path))

forecast_version_base_year_folder_location=r'{}\BASE_YEAR'.format(forecast_version_folder_location)

In [6]:
forecast_version = get_newest_date_file(fr'{forecast_version_folder_location}', '_forecast_2020_till_2050_jtmt')

df_2020 = pd.read_excel(fr"{forecast_version_folder_location}\BASE_YEAR\2020_jtmt_forcast_full_{forecast_version}.xlsx").set_index('Taz_num').sort_index()
df_jtmt_2050 = pd.read_excel(fr"{forecast_version_folder_location}\{forecast_version}_forecast_2020_till_2050_jtmt.xlsx").set_index('Taz_num').sort_index()
df_iplan_2050 = pd.read_excel(fr"{forecast_version_folder_location}\{forecast_version}_forecast_2020_till_2050_iplan.xlsx").set_index('Taz_num').sort_index()
df_bau_2050 = pd.read_excel(fr"{forecast_version_folder_location}\{forecast_version}_forecast_2020_till_2050_bau.xlsx").set_index('Taz_num').sort_index()

In [7]:
BaseProjections_version = get_newest_date_file(r'{}\JTMT'.format(forecast_version_folder_location), 'BaseProjections2025_')

years = ['2020', '2025', '2030','2035', '2040', '2045', '2050']

senarios = ['JTMT', 'IPLAN', 'BAU']

BaseProjections_jtmt_2020 = pd.read_excel(fr'{forecast_version_folder_location}\BASE_YEAR\BaseProjections2020_{BaseProjections_version}.xlsx')

for sen in senarios:
    for year in years[1:]:
        globals()[f'BaseProjections_{sen}_{year}'] = pd.read_excel(fr'{forecast_version_folder_location}\{sen}\BaseProjections{year}_{BaseProjections_version}_{sen.lower()}.xlsx')

In [8]:
df_jtmt_2050 = pd.merge(df_jtmt_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')
df_iplan_2050 = pd.merge(df_iplan_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')
df_bau_2050 = pd.merge(df_bau_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')

In [9]:
schooldists = pd.pivot_table(df_jtmt_2050, index=['SCHOOLDIST'], aggfunc='sum').index
schn_names = pd.pivot_table(df_iplan_2050, index=['SCHN_NAME'], aggfunc='sum').index
muni_hebs = pd.pivot_table(df_iplan_2050, index=['Muni_Heb'], aggfunc='sum').index
main_sectors = pd.pivot_table(df_iplan_2050, index=['main_secto'], aggfunc='sum').index

In [10]:
def add_total_numbers_by_type_row(output_df, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df[df[f'{column_input_name}'] == column_input_value] 
        values_list.append(len(val))

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [11]:
# חישוב מספר לפי סוג
def add_total_row(output_df, column_input_name, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if column_input_name in df.columns:
            val = df[f'{column_input_name}'].sum()
            values_list.append("{:,}".format(int(val)))
        else:
            values_list.append(np.nan)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [12]:
# בודק שאין מספר שחוזר פעמיים
def add_taz_num_is_unique_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df_2020.index.is_unique
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [13]:
# חישוב מספר א"ת
def add_taz_num_count_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = len(df.index)
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [14]:
def add_students_by_SCHOOLDIST(output_df, column_key_name, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = pd.pivot_table(df, index=[f'{column_input_name}'], aggfunc="sum")
        if column_key_name in val.columns:
            values_list.append(val[column_key_name][column_input_value])
        else:
            values_list.append(np.nan)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [15]:
# חישוב אחוזי גידול
def calculate_growth_percentage(current_total, previous_total):
    val = ((current_total - previous_total) / previous_total * 100.0)
    return val

In [16]:
def set_percentage_growth_value(previous, df, sector_type_filter, current_total):
    if previous == '2020':
            previous_total = df[f'{sector_type_filter}'].sum()
    else:
            previous_total = df[f'{sector_type_filter}_{previous}'].sum()
    val = calculate_growth_percentage(current_total, previous_total)
    return val

In [17]:
def add_percentage_growth_row(sector_type_filter, output_df):
        values_list = [np.nan]

        for previous, current in zip(years, years[1:]):
                for sen in senarios:
                        current_total = globals()[f'df_{sen.lower()}_2050'][f'{sector_type_filter}_{current}'].sum()
                        val = set_percentage_growth_value(previous, globals()[f'df_{sen.lower()}_2050'], sector_type_filter, current_total)
                        values_list.append(val)
                output_df.loc[f'percentage growth {sector_type_filter} {previous}-{current}'] = values_list
                values_list = [np.nan]
        return output_df

In [18]:
def add_percentage_growth_by_filter_row(filter, pivot_filter, sector_type_filter, output_df, row_title):
        values_list = [np.nan]

        for previous, current in zip(years, years[1:]):
                for i, item in enumerate(filter):
                        for sen in senarios:
                                df_sector = pd.pivot_table(globals()[f'df_{sen.lower()}_2050'], index=[pivot_filter], aggfunc="sum")
                                current_total = df_sector.iloc[i][f'{sector_type_filter}_{current}']
                                val = set_percentage_growth_value(previous, df_sector.iloc[i], sector_type_filter, current_total)
                                values_list.append(val)
                        output_df.loc[f'percentage growth {row_title} {filter[i]} {previous}-{current}'] = np.float32(values_list)
                        values_list = [np.nan]
        return output_df

In [19]:
monitoring_df = pd.DataFrame(columns=['2020', 'JTMT', 'iplan', 'bau'])

In [20]:
SCHOOLDIST_list = list(df_2020['SCHOOLDIST'].value_counts().index)
sectors_list = list(df_2020['main_secto'].value_counts().index)
zonetypes_list = list(df_2020['zonetype'].value_counts().index)
jeru_metros_list = list(df_2020['jeru_metro'].value_counts().index)
in_jerusals_list = list(df_2020['in_jerusal'].value_counts().index)

In [21]:
# אוכלוסיה
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva', 'pop_2020')
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva_2050', 'pop_2050')

# מועסקים
monitoring_df = add_total_row(monitoring_df, 'total_emp', 'emp_2020')
monitoring_df = add_total_row(monitoring_df, 'total_emp_2050', 'emp_2050')

# מספר א"ת לפי סקטור
for item in sectors_list:
    monitoring_df = add_total_numbers_by_type_row(monitoring_df, 'main_secto', item, f'Taz_nums for {item} sector')

# סטודנטים
monitoring_df = add_total_row(monitoring_df, 'univ', 'univ_2020')
monitoring_df = add_total_row(monitoring_df, 'univ_2050', 'univ_2050')

# בתי ספר
monitoring_df = add_total_row(monitoring_df, 'student', 'student_2020')
monitoring_df = add_total_row(monitoring_df, 'student_2050', 'student_2050')

# ישיבות
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim', 'student_yeshiva_and_kollim_2020')
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim_2050', 'student_yeshiva_and_kollim_2050')

# מספר zonetype
for item in zonetypes_list:
    monitoring_df = add_total_numbers_by_type_row(monitoring_df, 'zonetype', item, f'zonetype_{item}')

# מספר jeru_metro
for item in jeru_metros_list:
    monitoring_df = add_total_numbers_by_type_row(monitoring_df, 'jeru_metro', item, f'jeru_metro_{item}')

# מספר in_jerusal
for item in in_jerusals_list:
    monitoring_df = add_total_numbers_by_type_row(monitoring_df, 'in_jerusal', item, f'in_jerusal_{item}')

# אין א"ת שחוזר פעמיים
monitoring_df = add_taz_num_is_unique_row(monitoring_df, 'taz_num_is_unique')

# מספר א"ת
monitoring_df = add_taz_num_count_row(monitoring_df, 'taz_num_count')

In [22]:
values_list = [np.nan]

In [23]:
# גידול אוכלוסיה בכל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva', monitoring_df)

# גידול סטודנטים בכל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('univ', monitoring_df)

# גידול תלמידי ישיבה בכל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('student_yeshiva_and_kollim', monitoring_df)

# גידול אוכלוסיה בכל חומש בכל תרחיש לפי מגזר
monitoring_df = add_percentage_growth_by_filter_row(main_sectors, 'main_secto', 'pop_without_dorms_yeshiva', monitoring_df, 'sector')

C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)


In [24]:
def set_BaseProjections_previous_sector(previous):
    if previous == '2020':
        BaseProjections_previous_sector = pd.pivot_table(BaseProjections_jtmt_2020, index=['sector'], aggfunc="sum")
    else:
        BaseProjections_previous_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{previous}'], index=['sector'], aggfunc="sum")
    
    return BaseProjections_previous_sector

In [25]:
# גידול גילאים 15-24 באוכלוסיה החרדית בכל חומש בכל תרחיש
for previous, current in zip(years, years[1:]):
    for sen in senarios:
        BaseProjections_current_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{current}'], index=['sector'], aggfunc="sum")
        BaseProjections_previous_sector = set_BaseProjections_previous_sector(previous)

        previous_total = (BaseProjections_previous_sector.iloc[3]['age15_19']/ 5 * 1) + BaseProjections_previous_sector.iloc[3]['age20_24']
        BaseProjections_age15_24_U_previous_total = previous_total / 2

        current_total = (BaseProjections_current_sector.iloc[3]['age15_19']/ 5 * 1) + BaseProjections_current_sector.iloc[3]['age20_24']
        BaseProjections_age15_24_U_current_total = current_total / 2

        val = calculate_growth_percentage(BaseProjections_age15_24_U_current_total, BaseProjections_age15_24_U_previous_total)
        values_list.append(val)
    monitoring_df.loc[f'percentage growth U_Orthodox age18_24 {previous}-{current}'] = values_list
    values_list = [np.nan]

In [26]:
# גידול גילאים 20-29 בכל מגזר בכל חומש בכל תרחיש
for i, sector in enumerate(main_sectors):
    for previous, current in zip(years, years[1:]):
        for sen in senarios:
            BaseProjections_current_sector = pd.pivot_table(globals()[f'BaseProjections_{sen}_{current}'], index=['sector'], aggfunc="sum")
            BaseProjections_previous_sector = set_BaseProjections_previous_sector(previous)

            previous_total = (BaseProjections_previous_sector.iloc[i]['age20_24']/ 5 * 3) + BaseProjections_previous_sector.iloc[i]['age25_29']/ 5 * 4
            current_total = (BaseProjections_current_sector.iloc[i]['age20_24']/ 5 * 3) + BaseProjections_current_sector.iloc[i]['age25_29']/ 5 * 4

            val = calculate_growth_percentage(current_total, previous_total)
            values_list.append(val)
        monitoring_df.loc[f'percentage growth {sector} age20_29 {previous}-{current}'] = values_list
        values_list = [np.nan]

In [27]:
age6_18_list = [np.nan]
students_list = [np.nan]
demand_percentages_list = [np.nan]

for i, schooldist in enumerate(schooldists):
    for year in years:
        for sen in senarios:
            if year == '2020':
                globals()[f'BaseProjections_students_{sen}_2020'] = pd.merge(BaseProjections_jtmt_2020, df_2020[['SCHOOLDIST']],left_on='TAZ', right_on='Taz_num')
                globals()[f'BaseProjections_students_{sen}_2020'] = pd.pivot_table(globals()[f'BaseProjections_students_{sen}_2020'], index=['SCHOOLDIST'], aggfunc="sum")
            else:
                globals()[f'BaseProjections_students_{sen}_{year}'] = pd.merge(globals()[f'BaseProjections_{sen}_{year}'],df_2020[['SCHOOLDIST']],left_on='TAZ', right_on='Taz_num')
                globals()[f'BaseProjections_students_{sen}_{year}'] = pd.pivot_table(globals()[f'BaseProjections_students_{sen}_{year}'], index=['SCHOOLDIST'], aggfunc='sum')
            
            age6_9 = globals()[f'BaseProjections_students_{sen}_{year}']['age5_9']/5*4
            age15_18 = globals()[f'BaseProjections_students_{sen}_{year}']['age15_19']/5*4
            age6_18 = age6_9 + globals()[f'BaseProjections_students_{sen}_{year}']['age10_14'] + age15_18

            age6_18_list.append(age6_18.iloc[i])
            students_list.append(globals()[f'BaseProjections_students_{sen}_{year}']['student'].iloc[i])

            if globals()[f'BaseProjections_students_{sen}_{year}']['student'].iloc[i]!= 0:
                demand_percentages_list.append((age6_18.iloc[i] - globals()[f'BaseProjections_students_{sen}_{year}']['student'].iloc[i]) / globals()[f'BaseProjections_students_{sen}_{year}']['student'].iloc[i] * 100)
            else:
                demand_percentages_list.append(0)
            
        monitoring_df.loc[f'students at district {i} {year}'] = students_list
        monitoring_df.loc[f'age6_18 at district {i} {year}'] = age6_18_list
        monitoring_df.loc[f'demand percentages at district {i} {year}'] = demand_percentages_list

        age6_18_list = [np.nan]
        students_list = [np.nan]
        demand_percentages_list = [np.nan]

In [28]:
def add_total_sum_by_type_row(col_filters, type_filter, index):    
    values_list = [np.nan]

    for i, col_filter in enumerate(col_filters):
        for year in years:
            for sen in senarios:
                if year == '2020':
                    val = pd.pivot_table(df_2020, index=[index], aggfunc="sum")
                    values_list.append(val.iloc[i][f'{type_filter}'])
                else:
                    val = pd.pivot_table(globals()[f'df_{sen.lower()}_2050'], index=[index], aggfunc='sum')
                    values_list.append(val.iloc[i][f'{type_filter}_{year}'])
            monitoring_df.loc[f'{type_filter} at {col_filter} in {year}'] = values_list
            values_list = [np.nan]
    return monitoring_df

In [29]:
# חישוב מועסקים עוקב אוכלוסיה לפי שכונות בירושלים כל חומש בכל תרחיש
monitoring_df = add_total_sum_by_type_row(schn_names, 'emp_okev', 'SCHN_NAME')

# חישוב מועסקים עוקב אוכלוסיה לפי רשות כל חומש בכל תרחיש
monitoring_df = add_total_sum_by_type_row(muni_hebs, 'emp_okev', 'Muni_Heb')

# חישוב מועסקים לפי רשות כל חומש בכל תרחיש
monitoring_df = add_total_sum_by_type_row(muni_hebs, 'total_emp', 'Muni_Heb')

In [30]:
def add_pop_division_emp_okev_by_SCHN_NAME(col_filters, type_filter, index):    
    values_list = [np.nan]

    for i, col_filter in enumerate(col_filters):
        for year in years[:2]:
            for sen in senarios:
                if year == '2020':
                    val = pd.pivot_table(df_2020, index=[index], aggfunc="sum")
                    if val['emp_okev'].iloc[i] != 0:
                        values_list.append((val['pop_without_dorms_yeshiva'].iloc[i] / val['emp_okev'].iloc[i]))
                    else:
                        values_list.append(0)
                else:
                    val = pd.pivot_table(globals()[f'df_{sen.lower()}_2050'], index=[index], aggfunc='sum')
                    if val['emp_okev_2025'].iloc[i] != 0:
                        values_list.append(val['pop_without_dorms_yeshiva_2025'].iloc[i] / val['emp_okev_2025'].iloc[i])
                    else:
                        values_list.append(0)
            monitoring_df.loc[f'pop division {type_filter} at {col_filter} in {year}'] = values_list
            values_list = [np.nan]
    return monitoring_df

In [31]:
monitoring_df = add_pop_division_emp_okev_by_SCHN_NAME(schn_names, 'emp_okev', 'SCHN_NAME')

In [34]:
def division_total_emp_in_key(second_key, row_title):
    values_list = [np.nan]

    for year in years:
        for idx, df in enumerate([df_jtmt_2050, df_iplan_2050, df_bau_2050]):
            if year == '2020':
                if second_key == 'aprt':
                    value = df['total_emp'].sum() / df[fr'{second_key}_20'].sum()
                else:
                    value = df['total_emp'].sum() / df[second_key].sum()
                values_list.append(value)
            else:
                value = df[fr'total_emp_{year}'].sum() / df[fr'{second_key}_{year}'].sum()
                values_list.append(value)
        monitoring_df.loc[f'{row_title} division total_emp in {year}'] = values_list
        values_list = [np.nan]
    return monitoring_df

In [35]:
# מועסקים חלוקה באוכלוסייה
monitoring_df = division_total_emp_in_key('pop_without_dorms_yeshiva', 'pop')

# מועסקים חלוקה ביח"ד
monitoring_df = division_total_emp_in_key('aprt', 'aprt')

In [36]:
# גידול מועסקים בחינוך בכל חומש בכל תרחיש לפי שכונה
monitoring_df = add_percentage_growth_by_filter_row(schn_names, 'SCHN_NAME', 'emp_Education', monitoring_df, 'emp education')

# גידול מועסקים בחינוך בכל חומש בכל תרחיש לפי רשות
monitoring_df = add_percentage_growth_by_filter_row(muni_hebs, 'Muni_Heb', 'emp_Education', monitoring_df, 'emp education')

C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: invalid value encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: invalid value encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: invalid value encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: invalid value encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\Temp\ipykernel_27612\3179375671.py:3: RuntimeWarning: invalid value encountered in scalar divide
  val = ((current_total - previous_total) / previous_total * 100.0)
C:\Users\dpere\AppData\Local\T

In [37]:
# def highlight_cols(s): 
#     if s > 100:
#         color ='red'
#     elif s < -100:
#         color = 'red'
#     else: 
#         color = 'none'
#     return 'background-color: % s' % color

# idx = pd.IndexSlice[monitoring_df.index.get_level_values(0).str.contains('different'), :]
# monitoring_df = monitoring_df.style.applymap(highlight_cols, subset=idx)

In [38]:
monitoring_df.to_excel('monitoring_df_fixed_2.xlsx')
# monitoring_df